In [68]:
import pandas as pd

In [69]:
train = pd.read_csv('/home/kebler/Downloads/train.csv')
test = pd.read_csv('/home/kebler/Downloads/test.csv')
train.shape

(891, 12)

In [70]:
import matplotlib.pyplot as plt

In [71]:
def process_age(df, cutpoint, label_names):
    df["Age"] = df["Age"].fillna(-0.5)
    df["Age_categories"] = pd.cut(df["Age"], cutpoint, labels=label_names)
    return df

cut_points = [-1, 0, 5, 12, 18, 35, 60, 100]
label_names = ["Missing", "Infant", "Child", "Teenager", "Young_Adult", "Adult", "Senior"]
train = process_age(train, cut_points, label_names)
test = process_age(test, cut_points, label_names)


In [72]:
def create_dummies(df, column_name):
    dummies = pd.get_dummies(df[column_name], prefix=column_name)
    df = pd.concat([df, dummies], axis=1)
    return df

In [73]:
train = create_dummies(train, "Pclass")
test = create_dummies(test, "Pclass")
train = create_dummies(train, "Sex")
test = create_dummies(test, "Sex")
train = create_dummies(train, "Age_categories")
test = create_dummies(test, "Age_categories")

In [74]:
from sklearn.linear_model import LogisticRegression

In [75]:
columns = ['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
       'Age_categories_Missing','Age_categories_Infant',
       'Age_categories_Child', 'Age_categories_Teenager',
       'Age_categories_Young_Adult', 'Age_categories_Adult',
       'Age_categories_Senior']

In [76]:
holdout = test # from now on we will refer to this
               # dataframe as the holdout data

In [77]:
all_X = train[columns]
all_y = train["Survived"]

In [78]:
from sklearn.model_selection import cross_val_score
import numpy as np

In [79]:
lr = LogisticRegression(solver='lbfgs')
scores = cross_val_score(lr, all_X, all_y, cv=10)
np.mean(scores)

0.8024670865963002

In [80]:
lr.fit(all_X, all_y)
predictions = lr.predict(holdout[columns])
predictions

NotFittedError: This LogisticRegression instance is not fitted yet

In [67]:
holdout_ids = holdout["PassengerId"]
submission_df = {"PassengerId": holdout_ids, "Survived": predictions}
submission = pd.DataFrame(submission_df)
submission.to_csv('tsub.csv', index=False)